In [1]:
from os import listdir, path
from re import match
from csv import writer
from Network import NetworkGraph, NetworkFlow
from PathAssigning import MEPT

In [2]:
def extract_NetworkGraph(
    param_File
):
    NetworkGraphFile = param_File
    Graph = NetworkGraph()

    Line = NetworkGraphFile.readline()
    while Line:
        if Line.startswith("NODES"):
            break
        Line = NetworkGraphFile.readline()
    Line = NetworkGraphFile.readline()
    while False if match(r"^\)\s+$", Line) else True:
        Node = Line.strip().split(' ')[0]
        Graph.add_Node(Node)
        Line = NetworkGraphFile.readline()

    Line = NetworkGraphFile.readline()
    while Line:
        if Line.startswith("LINKS"):
            break
        Line = NetworkGraphFile.readline()
    Line = NetworkGraphFile.readline()
    while False if match(r"^\)\s+$", Line) else True:
        Splits = Line.strip().split(' ')
        Node1, Node2 = Splits[2], Splits[3]
        Bandwidth = float(Splits[5])
        Graph.add_Link(Node1, Node2, Bandwidth)
        Line = NetworkGraphFile.readline()

    return Graph

In [3]:
def extract_Flows(
    param_File
):
    FlowsFile = param_File
    Flows = []

    Line = FlowsFile.readline()
    while Line:
        if Line.startswith("DEMANDS"): break;
        Line = FlowsFile.readline()
    Line = FlowsFile.readline()
    while False if match(r"^\)\s+$", Line) else True:
        Splits = Line.strip().split(' ')
        SrcNode, DstNode = Splits[2], Splits[3]
        FlowRate = float(Splits[5]) * float(Splits[6])
        Flow = NetworkFlow(SrcNode, DstNode, FlowRate)
        Flows.append(Flow)
        Line = FlowsFile.readline()

    return Flows

In [4]:
def calc_EE(
    param_NetworkGraph
):
    Graph = param_NetworkGraph
    
    Links = Graph.get_Links()
    _numerator, _denominator = 0, len(Links)
    for Link in Links.values():
        State = Link[2]
        if State is True:
            _numerator += 1
    EE = 1 - _numerator / _denominator
    
    return EE

In [5]:
def find_UBounds(
    param_NetworkGraph,
    param_Flows,
    param_Index,
):
    Graph = param_NetworkGraph
    Flows = param_Flows
    EE = 0
    Umin = Umax = 0
    Index = param_Index

    _break = False
    for _index_Umin in range(1, 100, 1):
        _Umin = _index_Umin * 0.01
        
        for _index_Umax in range(100, _index_Umin, -1):
            _Umax = _index_Umax * 0.01
            Graph.reset_Network()
            try:
                MEPT(Graph, Flows, _Umin, _Umax)
            except:
                print(f"No path available for {Index}.")
                _break = True
                break
            _EE = calc_EE(Graph)
            if _EE >= EE:
                EE = _EE
                Umin, Umax = _Umin, _Umax
        if _break is True:
            break

    return Umin, Umax

In [6]:
def write_CSV(
    param_File,
    param_Nodes,
    param_Flows,
    param_Umin,
    param_Umax
):
    CSVFile = param_File
    Nodes = param_Nodes
    Flows = param_Flows
    Umin, Umax = param_Umin, param_Umax
    
    CSVFile.truncate(0)
    CSVWriter = writer(CSVFile)
    
    FlowData = []
    _initialrow = [0] * len(Nodes)
    for _index in range(len(Nodes)):
        FlowData.append(_initialrow.copy())
    for Flow in Flows:
        _srcindex = _dstindex = 0
        for _index in range(len(Nodes)):
            if Flow.SRC_NODE == Nodes[_index]:
                _srcindex = _index
                continue
            if Flow.DST_NODE == Nodes[_index]:
                _dstindex = _index
                continue
            
        FlowData[_srcindex][_dstindex] = Flow.FLOW_RATE
    
    CSVWriter.writerow(['{:.2f}'.format(round(Umin, 2)), '{:.2f}'.format(round(Umax, 2))])
    CSVWriter.writerows(FlowData)

In [ ]:
NetworkGraphFile = open("../public/data/raw_data/network-graph.txt", 'r')

Graph = extract_NetworkGraph(NetworkGraphFile)
Graph.initiate_AllPath()
Nodes = Graph.get_Nodes()

_index = 1
for FlowsFile in listdir("../public/data/raw_data/Flows"):
    FlowsFile = open("../public/data/raw_data/Flows/" + FlowsFile, 'r')
    Flows = extract_Flows(FlowsFile)
    FlowsFile.close()
    Umin, Umax = find_UBounds(Graph, Flows, _index)
    Graph.reset_Network()
    print(f"{_index}: {Umin} - {Umax}\n")
    CSVFile = open(path.join("../public/data/processed_data", _index.__str__() + ".csv"), 'w', newline = '')
    write_CSV(CSVFile, Nodes, Flows, Umin, Umax)
    CSVFile.close()
    _index += 1

NetworkGraphFile.close()

1: 0.07 - 0.65
2: 0.1 - 0.66
3: 0.13 - 0.7000000000000001
4: 0.07 - 0.9400000000000001
5: 0.06 - 0.6
6: 0.01 - 0.64
7: 0.01 - 0.88
8: 0.01 - 0.71
9: 0.01 - 0.64
10: 0.03 - 0.9500000000000001
11: 0.01 - 0.73
12: 0.01 - 0.79
13: 0.01 - 0.6900000000000001
14: 0.02 - 0.8
15: 0.03 - 0.49
16: 0.09 - 0.9
17: 0.07 - 0.8300000000000001
18: 0.07 - 0.92
19: 0.04 - 0.89
20: 0.1 - 0.8200000000000001
21: 0.03 - 0.63
22: 0.01 - 0.72
23: 0.06 - 0.72
24: 0.09 - 0.84
25: 0.01 - 0.6900000000000001
26: 0.01 - 0.92
27: 0.02 - 0.64
28: 0.02 - 0.89
29: 0.01 - 0.8300000000000001
30: 0.01 - 0.78
31: 0.06 - 0.9
32: 0.05 - 0.61
33: 0.04 - 0.68
34: 0.04 - 0.46
35: 0.08 - 0.7000000000000001
36: 0.06 - 0.77
37: 0.01 - 0.76
38: 0.02 - 0.8300000000000001
39: 0.01 - 0.92
40: 0.1 - 0.78
41: 0.02 - 0.87
42: 0.01 - 0.9
43: 0.1 - 0.81
44: 0.05 - 0.8
45: 0.15 - 0.8200000000000001
46: 0.04 - 0.91
47: 0.05 - 0.7000000000000001
48: 0.07 - 0.86
49: 0.21 - 0.8
50: 0.02 - 0.86
51: 0.02 - 0.67
52: 0.03 - 0.9
53: 0.07 - 0.8
54: 0.